In [1]:
from fastai.vision.all import *
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

If the GPU is available use it for the computation otherwise use the CPU.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## Fashion MNIST

The Fashion MNIST dataset contains thousands of grayscale images of different types of clothes. There are ten distinct categories encoded into numbers 0 - 10:

0   T-shirt/top

1	Trouser

2	Pullover

3	Dress

4	Coat

5	Sandal

6	Shirt

7	Sneaker

8	Bag

9	Ankle boot

Download the dataset

In [ ]:
train_set = datasets.FashionMNIST("./data", download=True, train=True, transform=transforms.Compose([transforms.ToTensor(), torch.flatten]))
test_set = datasets.FashionMNIST("./data", download=True, train=False, transform=transforms.Compose([transforms.ToTensor(), torch.flatten]))

We are going to feed the dataset into our network in batches of size 64 as the dataset is too big to be used all at once.

In [4]:
data_loader = DataLoaders.from_dsets(train_set, test_set, bs=128)

Let's check the mini batch you'll get from the train data loader:

In [ ]:
x, y = next(iter(data_loader.train))
x.size(), y.size()

In [ ]:
len(train_set)

In [ ]:
len(test_set)

In [ ]:
train_set[0][0].size()

In [9]:
clothes_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                  'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Let's plot some images from our dataset

In [ ]:
def plot_image(x, title = ""):
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(torch.reshape(x, (28,28)), cmap=plt.cm.binary)
  plt.xlabel(title)

plt.figure(figsize=(8,8))
for i in range(0, 160, 10):
  plt.subplot(4, 4, i // 10 + 1)
  plot_image(train_set[i][0], clothes_labels[train_set[i][1]])

# Dense NN

Let's try to apply dense neural network (we talked about it in the previous lesson) to our images.

We'll use a single layer network. All layers are Linear - the classic fully-connected neural network layers.

- The hidden layer will have 512 units using the ReLU activation function.
- The output layer will have 10 units, corresponding to the 10 classes, and use softmax function.

The general convention for Pytorch network classes is that you create all your layers in the constructor, and then lay out their relationship in the `forward()` method.

In [13]:
class Net(nn.Module):
    def __init__(self, image_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(image_size, _) # TODO nn.Linear(n_input_neurons, n_output_neurons)
        self.fc2 = nn.Linear(_, _) # TODO
        self.fc3 = nn.Linear(_, _) # TODO

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [ ]:
model = Net(image_size=_) # TODO
model.to(device)
model

## Training

In [15]:
# learning rate
lr = 5e-4
# loss function
loss_func = nn.CrossEntropyLoss()

Rather then setting up the model optimization in Pytorch, we are going to use fastai's Learner object. (By default, it uses the Adam optimizer.)

In [16]:
learn = Learner(data_loader, model, loss_func=loss_func, metrics=accuracy)

In [ ]:
learn.fit(n_epoch=3, lr=lr)

In [ ]:
learn.recorder.plot_loss()



---



# CNN

Let's try a different architecture, this time a Convolutional Neural Network

Load data in different format

In [18]:
train_set = datasets.FashionMNIST("./data", download=True, train=True, transform=transforms.Compose([transforms.ToTensor()]))
test_set = datasets.FashionMNIST("./data", download=True, train=False, transform=transforms.Compose([transforms.ToTensor()]))

data_loader = DataLoaders.from_dsets(train_set, test_set, bs=128)

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=24*24*64, out_features=128)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.softmax(self.fc2(x), dim=1)
        return x

In [ ]:
model = CNN()
model.to(device)
model

### Training

In [21]:
# learning rate
lr = 5e-4
# loss function
loss_func = nn.CrossEntropyLoss()

Rather then setting up the model optimization in Pytorch, we are going to use fastai's Learner object. (By default, it uses the Adam optimizer.)

In [22]:
learn = Learner(data_loader, model, loss_func=loss_func, metrics=accuracy)

In [ ]:
learn.fit(3, lr)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
plot_image(test_set[0][0])

In [ ]:
clothes_labels[torch.argmax(learn.predict(test_set[0])[0])]